In [59]:
import netCDF4 as nc4
import xarray as xr
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
sys.path.append('/glade/u/home/adamhb/Earth-System-Model-Tools/')
import importlib
#importlib.reload(fields)
from fields import fields
import esm_tools
from matplotlib import pyplot as plt
import os

## Paths and constants

In [128]:
case_name = 'CZ2_equilibrium_040924_-17e2acb6a_FATES-1449c787'
s_per_year = 3.154e+7
fates_litterclass_name = ["twig","small_branch","large_branch","trunk","dead_leaves","live_grass"]
np.set_printoptions(suppress=True)

In [124]:
def get_litter_out(ds):
    '''
    returns litter out in kg m-2 s-1
    '''
    return ds['FATES_LITTER_OUT'].mean(dim = "time").values * s_per_year

def get_frag_scaler(inst_tag):
    ds = esm_tools.load_fates_output_data(model_output_root='/glade/derecho/scratch/adamhb',
                                      case_name=case_name,years = list(range(1849,1869)),
                                      fields=['FATES_LITTER_OUT','FATES_FRAGMENTATION_SCALER_SL'],
                                      inst_tag=inst_tag)
    
    return ds['FATES_FRAGMENTATION_SCALER_SL'].mean(dim = "time").isel(levsoi = 0).values

def get_max_frag_rate(inst_tag):
    path_to_param_files = '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/CZ2_equilibrium_040924/'
    ref_nc_file = esm_tools.find_files_with_substring(directory=path_to_param_files,
                                                    substring=inst_tag)
            
    # Get full path of ref param file
    ref_nc_file_full_path = os.path.join(path_to_param_files,ref_nc_file[0])
    print(ref_nc_file_full_path)
    max_frag_rates = [esm_tools.extract_variable_from_netcdf(ref_nc_file_full_path,"fates_frag_maxdecomp",i) for i in list(range(0,6))]
    return np.array(max_frag_rates)

def get_bias(inst_tag, frag_obs = frag_obs_mean):
    max_frag = get_max_frag_rate(inst_tag)
    f_scaler = f_scaler_mean
    pred_frag_rate = max_frag * f_scaler
    bias = pred_frag_rate - frag_obs
    return np.concatenate((max_frag,bias),axis = 0)

## Obs of fragmentation rates

In [122]:
fates_litterclass_name = ["twig","small_branch","large_branch","trunk","dead_leaves","live_grass"]
frag_obs_mean = np.array([0.1,# Based on Campbell et al., 2006 and McIntyre and Ottman, 2006
                 0.09,# Based on Campbell et al., 2006 and McIntyre and Ottman, 2006
                 0.08,# Based on Campbell et al., 2006 and McIntyre and Ottman, 2006
                 0.04,# McIntyre and Ottman, 2006 and Harmon et al., 2020
                 0.16, # Based on Hart et al., 1992 and McIntyre and Ottman, 2006
                 0.16])
frag_obs_mean

array([0.1 , 0.09, 0.08, 0.04, 0.16, 0.16])

## Get mean frag scaler from prior runs

In [85]:
f_scaler = []
for tag in esm_tools.inst_to_tag(list(range(1,20))):
    tmp = get_frag_scaler(tag)
    print(tmp)
    f_scaler.append(tmp)

f_scaler = [f_scaler[i].item() for i in list(range(len(f_scaler)))]
f_scaler_mean = np.array(f_scaler).mean()

-- your data have been read in -- 
0.14193371
-- your data have been read in -- 
0.13646555
-- your data have been read in -- 
0.13256435
-- your data have been read in -- 
0.12826414
-- your data have been read in -- 
0.1335383
-- your data have been read in -- 
0.12980542
-- your data have been read in -- 
0.1427188
-- your data have been read in -- 
0.12542418
-- your data have been read in -- 
0.13799712
-- your data have been read in -- 
0.12889846
-- your data have been read in -- 
0.13203765
-- your data have been read in -- 
0.12737763
-- your data have been read in -- 
0.12697484
-- your data have been read in -- 
0.124168165
-- your data have been read in -- 
0.12928279
-- your data have been read in -- 
0.13863823
-- your data have been read in -- 
0.12542655
-- your data have been read in -- 
0.14350632
-- your data have been read in -- 
0.124387994


## Find best params

In [149]:
def get_optimal_max_decomp_param(f_scaler,obs_low, obs_high):
    optimal_param_low = obs_low / f_scaler
    optimal_param_high = obs_high / f_scaler
    print("low",optimal_param_low)
    print("high",optimal_param_high)
    return 

obs_low = frag_obs_mean * 0.5
obs_high = frag_obs_mean * 1.5

["twig","small_branch","large_branch","trunk","dead_leaves","live_grass"]
get_optimal_max_decomp_param(f_scaler_mean,obs_low, obs_high)

low [0.37857501 0.34071751 0.30286001 0.15143001 0.60572002 0.60572002]
high [1.13572504 1.02215254 0.90858003 0.45429002 1.81716007 1.81716007]


In [135]:
twig = frag_obs_mean[0]
frag_obs_mean_frac_of_twig = frag_obs_mean/twig
frag_obs_mean_frac_of_twig

array([1. , 0.9, 0.8, 0.4, 1.6, 1.6])

In [151]:
frag_obs_mean

array([0.1 , 0.09, 0.08, 0.04, 0.16, 0.16])

In [146]:
f_scaler_mean

0.13207422079224335

In [147]:
0.1893 * f_scaler_mean

0.025001649995971664

In [133]:
get_max_frag_rate('0020')

/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles/CZ2_equilibrium_040924/ca_5pfts_100523_0020.nc


array([  1.57454574,   1.1597135 ,   1.1597135 ,   0.19      ,
         3.02797257, 999.        ])

In [ ]:
f_scaler_mean

In [79]:
decay_loc = pd.read_csv('~/california-fates/cwd_decay/decay_rate_location.csv')
decay_data = pd.read_csv('~/california-fates/cwd_decay/decay_rates.csv')

In [82]:
refs = decay_loc.loc[decay_loc['LOCATION'].isin(['US - CA','US - OR'])]['REFERENCE']
decay_data.loc[decay_data['Reference'].isin(refs)]

,Genus,Species,subspecies,Family,Climate zone,Forest type,Positon,Diameters (cm),Model type,Target variable,k (per year),stderr k (per year),Decay resistance,Initial density (Mg per m cubed),Reported precipitation (mm per year),Reported temperature ( C ),WC precipitation (mm per year),WC temperature ( C ),Reference,Unnamed: 19


In [70]:
import pandas as pd

# Example data setup
data = {
    'LOCATION': ['US - OR', 'US - WA', 'US - WY', 'US - NH', 'US - CA'],
    'FOREST_TYPE': ['EF', 'EF', 'EF', 'EF', 'EF'],
    'LATITUDE': ['43.03 N', '43.25 N', '41 N', '44.01 N', '43.73 N'],
    'LONGITUDE': ['124.30 W', '124.15 W', '106 W', '71.48 W', '121.64 W'],
    'REFERENCE': [16, 23, 25, 26, 30]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Convert LATITUDE and LONGITUDE to numeric after removing 'N' and 'W' and converting to negative longitudes
df['LATITUDE'] = df['LATITUDE'].str.replace(' N', '').astype(float)
df['LONGITUDE'] = df['LONGITUDE'].str.replace(' W', '').astype(float) * -1

# Define bounds for the Sierra Nevada region
lat_min = 36.0
lat_max = 39.5
lon_min = -120.0
lon_max = -118.0

# Filter for Sierra Nevada in California
sierra_nevada_df = df[(df['LOCATION'] == 'US - CA') &
                      (df['LATITUDE'] >= lat_min) &
                      (df['LATITUDE'] <= lat_max) &
                      (df['LONGITUDE'] >= lon_min) &
                      (df['LONGITUDE'] <= lon_max)]

print(sierra_nevada_df)


Empty DataFrame
Columns: [LOCATION, FOREST_TYPE, LATITUDE, LONGITUDE, REFERENCE]
Index: []


In [ ]:
decay_loc